<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/BTC_G%C3%96RE_ROC_DE%C4%9EER%C4%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

# Binance API'sinden veri almak için URL
# CoinGecko API'yi kullanacağız çünkü Binance API'si için API anahtarı gereklidir.
api_url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"

# Binance'teki tüm coin'leri alıyoruz
response = requests.get(api_url)
data = response.json()

# Coin'lerin fiyat verilerini çıkaralım
coins = {}

# Binance'teki tüm coin'leri dolaşalım
for coin_data in data:
    coin_id = coin_data['id']
    coin_name = coin_data['symbol'].upper()
    current_price = coin_data['current_price']
    previous_price = coin_data['high_24h']  # 24 saatlik yüksek fiyatı, bir referans olarak alıyoruz

    # Eğer coin'in fiyatı varsa, ROC hesaplamak için ekliyoruz
    if current_price and previous_price:
        coins[coin_name] = {'current_price': current_price, 'previous_price': previous_price}

# BTC için ROC hesaplaması yapalım
btc_data = [coin for coin in data if coin['id'] == 'bitcoin'][0]
btc_roc = ((btc_data['current_price'] - btc_data['high_24h']) / btc_data['high_24h']) * 100

# ROC hesaplamak için fonksiyon
def calculate_roc(current_price, previous_price):
    return ((current_price - previous_price) / previous_price) * 100

# ROC değerlerini hesaplayalım
roc_values = {}

for coin, prices in coins.items():
    roc_values[coin] = calculate_roc(prices['current_price'], prices['previous_price'])

# Sonuçları DataFrame olarak gösterebiliriz
roc_df = pd.DataFrame(list(roc_values.items()), columns=['Coin', 'ROC (%)'])
roc_df['ROC (%)'] = roc_df['ROC (%)'].round(2)  # Yüzdeyi iki ondalıklı olarak yuvarlayalım

# BTC'nin ROC değerini aşan coin'leri bulalım
coins_above_btc_roc = roc_df[roc_df['ROC (%)'] > btc_roc]

# Sonuçları yazdıralım
print("Tüm Coin'lerin ROC Değerleri:")
print(roc_df)
print("\nBTC ROC Değerini Geçen Coin'ler:")
print(coins_above_btc_roc)


Tüm Coin'lerin ROC Değerleri:
       Coin  ROC (%)
0       BTC    -2.93
1       ETH    -3.50
2       XRP    -3.98
3      USDT    -0.06
4       SOL    -7.61
..      ...      ...
94     NEXO    -2.90
95     MSOL    -7.55
96  SOLVBTC    -3.19
97    JASMY    -7.33
98     USD0    -0.30

[99 rows x 2 columns]

BTC ROC Değerini Geçen Coin'ler:
     Coin  ROC (%)
0     BTC    -2.93
3    USDT    -0.06
5     BNB    -2.14
7    USDC     0.00
15   WBTC    -2.84
16    TON    -2.80
22    LEO    -0.21
25    BGB    -2.08
29   USDS    -0.42
33   USDE    -0.26
37    XMR    -0.74
38    WBT    -0.70
40    ETC    -2.58
46    DAI    -0.12
55     GT    -2.53
73  FDUSD    -0.40
75   LBTC    -2.88
78    FTN    -0.51
88  SUSDS    -0.96
90    KCS    -1.48
94   NEXO    -2.90
98   USD0    -0.30
